# Vital Parks Script Part 2

## This is the part of the vital parks script that gathers and formats the location data for each subcategory.

## This involves creating Location-Display and Location_Access tables for each sub-category

# Import Packages

In [1]:
import networkx as nx
import pandas as pd 
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon, MultiPolygon
from shapely.geometry import Polygon, Point
import requests
import zipfile
import io

# Get Locations

## Import Structures and Allsites Tables to match some locations

### Public Data

In [2]:
allsites_sql = ('''https://data.cityofnewyork.us/resource/buk3-3qpr.geojson?$select=prop_id,multipolygon&$limit=50000''')
allsites = gpd.read_file(allsites_sql).to_crs(2263)

In [3]:
funcional_groups_sql = ('''https://data.cityofnewyork.us/resource/xhvt-s4va.geojson?$select=gispropnum,multipolygon&$limit=50000''')
funcional_groups = gpd.read_file(funcional_groups_sql).to_crs(2263)
funcional_groups = funcional_groups.rename(columns={'gispropnum':'prop_id'})
funcional_groups = funcional_groups[funcional_groups['prop_id'].str.contains('Group')]

In [4]:
allsites = pd.concat([allsites,funcional_groups])

In [5]:
Structures_sql = ('''https://data.cityofnewyork.us/resource/n8q6-i44s.geojson?$limit=50000''')
Structures = gpd.read_file(Structures_sql).to_crs(2263)

## Format and upload Locations for each category (Sometimes access/display can be different)

In [6]:
Category_locations_raw = {}

Category_locations_for_display = {}
Category_locations_for_access = {}

## Athletic Facilities

In [7]:
current_category_type = 'Athletic_Facility'

###################################################
# Import Current athletic facilities
###################################################
athletic_facilities_sql = ('''https://data.cityofnewyork.us/resource/qnem-b8re.geojson?$select=featurestatus,primary_sport,multipolygon&$limit=50000 ''')
athletic_facilities_raw = gpd.read_file(athletic_facilities_sql).to_crs(2263)
athletic_facilities_raw = athletic_facilities_raw[athletic_facilities_raw['featurestatus'].isin(['Active','Closed Temporarily'])]


Category_locations_for_display[current_category_type] = athletic_facilities_raw.reset_index(drop=True)

Category_locations_for_display[current_category_type]['Category'] = current_category_type
Category_locations_for_display[current_category_type]['Future'] = 'No'
Category_locations_for_display[current_category_type] = Category_locations_for_display[current_category_type].rename(columns={'primary_sport':'Feature'})

Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='HDB','Feature'] = 'Handball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='BKB','Feature'] = 'Basketball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='TNS','Feature'] = 'Tennis'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='SFB','Feature'] = 'Softball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='SCR','Feature'] = 'Soccer'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='BSB','Feature'] = 'Baseball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='VLB','Feature'] = 'Volleyball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='TRK','Feature'] = 'Track'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='FTB','Feature'] = 'Football'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='MPPA','Feature'] = 'Multipurpose'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='BOC','Feature'] = 'Bocce'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='CRK','Feature'] = 'Cricket'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='PKB','Feature'] = 'Pickleball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='HKY','Feature'] = 'Hockey'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='NTB','Feature'] = 'Netball'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='RBY','Feature'] = 'Rugby'
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature']=='FFB','Feature'] = 'Flagfootball'

#Fix the two entries that dont have a primary sport
Category_locations_for_display[current_category_type].loc[Category_locations_for_display[current_category_type]['Feature'].isna(),'Feature']='Handball'
Category_locations_for_display[current_category_type].loc[:,'SubCategoryName'] = 'Cat:Athletic_Facility$Feature:'+Category_locations_for_display[current_category_type]['Feature'].astype(str) +'$Future:No'
Category_locations_for_display[current_category_type] = Category_locations_for_display[current_category_type][['Category','SubCategoryName','Feature','Future','geometry']]


##################################################
# Import Current Skateparks
##################################################
skate_park_current_sql = ('''https://data.cityofnewyork.us/resource/8knb-zsb6.geojson''')
skate_park_current = gpd.read_file(skate_park_current_sql).to_crs(2263)

skate_park_current['Category'] = 'Athletic_Facility'
skate_park_current['SubCategoryName'] = 'Cat:Athletic_Facility'+'$Feature:Skate_Park'+'$Future:No'
skate_park_current['Feature'] = 'Skate_Park'
skate_park_current['Future'] = 'No'

skate_park_current = skate_park_current[['Category','SubCategoryName','Feature','Future','geometry']]
###################################################
###################################################


###################################################
# Concat display locations for athletic facilities with display locations for Skateparks
###################################################
Category_locations_for_display[current_category_type] = pd.concat([Category_locations_for_display[current_category_type],skate_park_current]).reset_index(drop=True)
###################################################
###################################################


###################################################
# Define access locations to be the same as display locations for athletic facilities
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Community Gardens (GreenThumb)

In [8]:
current_category_type = 'GreenThumb'

###################################################
# Import Current Greenthumb community gardens
###################################################
Greenthumb_sql = ('''https://data.cityofnewyork.us/resource/p78i-pat6.geojson?$select=gardenname,status,multipolygon&$limit=50000''')
Greenthumb_current = gpd.read_file(Greenthumb_sql).to_crs(2263)

Greenthumb_current = Greenthumb_current[Greenthumb_current['status'].isin(['Active','Active (Unlicensed)'])]

current_category_name = 'Cat:'+current_category_type +'$'+'Future:No'

Greenthumb_current['Category'] = current_category_type
Greenthumb_current['SubCategoryName'] = current_category_name
Greenthumb_current['Future'] = 'No'

Category_locations_for_display[current_category_type] = Greenthumb_current[['Category','SubCategoryName','Future','geometry']].reset_index(drop=True)
####################################################
####################################################


###################################################
# Define access locations to be the same as display locations for Greenthumb community gardens
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Cooling Features

In [ ]:
current_category_type = 'Cooling_Feature'

###################################################
# Import Drinking Fountains
###################################################
Drinking_fountain_sql = ('''https://data.cityofnewyork.us/resource/qnv7-p7a2.geojson?$limit=50000''')
Drinking_fountain = gpd.read_file(Drinking_fountain_sql).to_crs(2263)
Drinking_fountain = Drinking_fountain[Drinking_fountain['featuresta'].isin(['Active','Closed Temporarily'])]
Drinking_fountain = Drinking_fountain[Drinking_fountain['position']!='Indoor']

current_category_name = 'Cat:Cooling_Feature'+'$Feature:Drinking_Fountain'

Drinking_fountain['Category'] = current_category_type
Drinking_fountain['SubCategoryName'] = current_category_name
Drinking_fountain['Feature'] = 'Drinking_Fountain'

Category_locations_for_display[current_category_type] = Drinking_fountain[['Category','SubCategoryName','Feature','geometry']]
###################################################
###################################################


###################################################
# Import Spray Showers
###################################################
spray_shower_sql = ('''https://data.cityofnewyork.us/resource/ckaz-6gaa.geojson?$limit=50000''')
spray_shower = gpd.read_file(spray_shower_sql).to_crs(2263)

current_category_name = 'Cat:Cooling_Feature'+'$Feature:Spray_Shower'

spray_shower['Category'] = current_category_type
spray_shower['SubCategoryName'] = current_category_name
spray_shower['Feature'] = 'spray_shower'

spray_shower = spray_shower[['Category','SubCategoryName','Feature','geometry']]
###################################################
###################################################

Category_locations_for_display[current_category_type] = pd.concat([Category_locations_for_display[current_category_type],spray_shower]).reset_index(drop=True)


###################################################
# Define access locations to be the same as display locations for Cooling Features
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################


## Dog Runs

In [10]:
current_category_type = 'Dog_Run'

###################################################
# Import Current Dog Runs
###################################################
dog_run_sql = ('''https://data.cityofnewyork.us/resource/hxx3-bwgv.geojson?$select=name,featurestatus,the_geom&$limit=50000''')
dog_run_current = gpd.read_file(dog_run_sql).to_crs(2263)
dog_run_current = dog_run_current[dog_run_current['featurestatus'].isin(['Active','Closed Temporarily'])]

current_category_name = 'Cat:Dog_Run'+'$Future:No'

dog_run_current['Category'] = current_category_type
dog_run_current['SubCategoryName'] = current_category_name
dog_run_current['Future'] = 'No'

Category_locations_for_display[current_category_type] = dog_run_current[['Category','SubCategoryName','Future','geometry']]
###################################################
###################################################

###################################################
# Define access locations to be the same as display locations for DogRuns
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Forever Wild

In [11]:
current_category_type = 'Forever_Wild'

###################################################
# Import Forever Wild Sites
###################################################

forever_wild_sql = ('''https://data.cityofnewyork.us/resource/48va-85tp.geojson?$select=propertyname,shape&$limit=50000''')
forever_wild_sites = gpd.read_file(forever_wild_sql).to_crs(2263)

current_category_name = 'Cat:Forever_Wild'

forever_wild_sites['Category'] = current_category_type
forever_wild_sites['SubCategoryName'] = current_category_name

Category_locations_for_display[current_category_type] = forever_wild_sites[['Category','SubCategoryName','geometry']]
###################################################
###################################################

###################################################
# Define access locations to be the same as display locations for Forever Wild
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Greenways

### No Public Data?

## Trails

In [12]:
current_category_type = 'Trails'

###################################################
# Import Current Trails
###################################################

Trails_sql = ('''https://data.cityofnewyork.us/resource/vjbm-hsyr.geojson?$limit=50000''')
Trails = gpd.read_file(Trails_sql).to_crs(2263)

Trails.loc[:,'Future']='No'

current_category_name = 'Cat:Trails'+'$Future:'+Trails['Future'].astype(str)

Trails['Category'] = current_category_type
Trails['SubCategoryName'] = current_category_name
Trails['Future']='No'

Trails = Trails[['Category','SubCategoryName','Future','geometry']]
###################################################
###################################################

Category_locations_for_display[current_category_type] = Trails
###################################################
###################################################

###################################################
# Define access locations to be the same as display locations for Trails
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Nature Centers

### No Public Data

In [13]:
# current_category_type = 'Nature_Center'

# ###################################################
# # Import Nature Centers
# ###################################################

# Nature_Centers = Structures[Structures['StructureFunction']=='Nature Center'].reset_index(drop=True)
# Nature_Centers = Nature_Centers[Nature_Centers['FEATURESTATUS'].isin(['Active','Closed Temporarily'])]
# Nature_Centers = Nature_Centers.rename_geometry('geometry')

# current_category_name = 'Cat:Nature_Center'

# Nature_Centers['Category'] = current_category_type
# Nature_Centers['SubCategoryName'] = current_category_name

# Category_locations_for_display[current_category_type] = Nature_Centers[['Category','SubCategoryName','geometry']]
# ###################################################
# ###################################################

# ###################################################
# # Define access locations to be the same as display locations for Nature Centers
# ###################################################
# Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
# ###################################################
# ###################################################

## Capital

In [14]:
current_category_type = 'Capital'

###################################################
# Import Completed and Future Capital Projects
###################################################

all_capital_sql = ('''https://data.cityofnewyork.us/resource/4hcv-tc5r.json?$select=totalfunding,currentphase,parkid,constructionactualcompletion&$limit=50000''')
all_capital = pd.read_json(all_capital_sql)

all_capital.loc[all_capital['totalfunding'].str.contains('Between'),'totalfunding'] = '1000000'
all_capital.loc[all_capital['totalfunding'].str.contains('Greater'),'totalfunding'] = '10000000'
all_capital.loc[all_capital['totalfunding'].str.contains('Less'),'totalfunding'] = '0'
all_capital['totalfunding'] = all_capital['totalfunding'].str.replace(',','').str.replace('$','')
all_capital['totalfunding'] = all_capital['totalfunding'].astype(int)
all_capital.loc[all_capital['totalfunding']>=400000,:]

complete_capital = all_capital[all_capital['currentphase'] == 'completed']
complete_capital = complete_capital.loc[(pd.Timestamp.now()-pd.to_datetime(complete_capital['constructionactualcompletion']))<pd.Timedelta(365*5,'days'),:]
complete_capital = complete_capital['parkid'].unique()
future_capital = all_capital[all_capital['currentphase']!='completed']['parkid'].unique()

completed_capital_sites = allsites.loc[allsites['prop_id'].isin(complete_capital),:].copy()
completed_capital_sites.loc[:,'Future'] = 'No'
completed_capital_sites = completed_capital_sites[['geometry','Future']]

future_capital_sites = allsites.loc[allsites['prop_id'].isin(future_capital),:].copy()
future_capital_sites.loc[:,'Future'] = 'Yes'
future_capital_sites = future_capital_sites[['geometry','Future']]
Capital_projects = pd.concat([completed_capital_sites,future_capital_sites]).reset_index(drop=True)

current_category_name = 'Cat:Capital'+'$Future:'+Capital_projects['Future'].astype(str)

Capital_projects['Category'] = current_category_type
Capital_projects['SubCategoryName'] = current_category_name

Category_locations_for_display[current_category_type] = Capital_projects[['Category','SubCategoryName','Future','geometry']]

###################################################
# Define access locations to be the same as display locations for capital projects
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################


## Engaged Partner Groups

In [15]:
current_category_type = 'Engaged_Partner'

###################################################
# Import partner group locations
###################################################

partner_groups_sql = ('''https://www.nycgovparks.org/bigapps/DPR_VolunteeringGroups_001.json?$limit=50000''')
partner_groups = pd.read_json(partner_groups_sql)
partner_groups = gpd.GeoDataFrame(partner_groups, geometry=gpd.points_from_xy(partner_groups['lng'], partner_groups['lat']), crs="EPSG:4326")
partner_groups = partner_groups.to_crs(2263)

current_category_name = 'Cat:'+current_category_type

partner_groups['Category'] = current_category_type
partner_groups['SubCategoryName'] = current_category_name

Category_locations_for_display[current_category_type] = partner_groups[['Category','SubCategoryName','geometry']]

###################################################
# Define access locations to be the same as display locations for partner groups
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## PEP

### No Public Data

## Parks Score

### No Public Data

## Playgrounds

In [16]:
current_category_type = 'Playgrounds'

###################################################
# Import current and future playgrounds
###################################################
playgrounds_sql = ('''https://data.cityofnewyork.us/resource/j55h-3upk.geojson?$limit=50000''')
playgrounds = gpd.read_file(playgrounds_sql).to_crs(2263)

current_category_name = 'Cat:Playgrounds'+'$Future:No'

playgrounds['Category'] = current_category_type
playgrounds['SubCategoryName'] = current_category_name
playgrounds['Future'] = 'No'

Category_locations_for_display[current_category_type] = playgrounds[['Category','SubCategoryName','Future','geometry']].reset_index(drop=True)

###################################################
# Define access locations to be the same as display locations for playgrounds
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Education and Enrichment

In [17]:
current_category_type = 'Education_Enrichment'

###################################################
# Import past and future public education and enrichment events
###################################################
Past_Year_Public_Events_sql = ('''https://www.nycgovparks.org/bigapps/DPR_PastYearEventCounts_001.json''')
Past_Year_Public_Events = pd.read_json(Past_Year_Public_Events_sql)
Past_Year_Public_Events['Future'] = 'No'

Future_Public_Events_sql = ('''https://www.nycgovparks.org/bigapps/DPR_UpcomingEventCounts_001.json''')
Future_Public_Events = pd.read_json(Future_Public_Events_sql)
Future_Public_Events['Future']='Yes'

Public_Events = pd.concat([Past_Year_Public_Events,Future_Public_Events]).reset_index(drop=True)
Public_Events = Public_Events.merge(allsites, left_on='propID', right_on='prop_id')
Public_Events = gpd.GeoDataFrame(Public_Events, geometry='geometry', crs=2263)

current_category_name = 'Cat:Education_Enrichment'+'$Future:'+Public_Events['Future'].astype(str)

Public_Events['Category'] = current_category_type
Public_Events['SubCategoryName'] = current_category_name

Category_locations_for_display[current_category_type] = Public_Events[['Category','SubCategoryName','Future','geometry']]
###################################################
###################################################


###################################################
# Define access locations to be the same as display locations for education and enrichment events
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Public Restrooms

In [18]:
current_category_type = 'Public_Restroom'

###################################################
# Import Current Public Restrooms
###################################################

current_public_restrooms = Structures[Structures['public_restroom']]
current_public_restrooms = current_public_restrooms[current_public_restrooms['featurestatus'].isin(['Active','Closed Temporarily'])]

current_category_name = 'Cat:'+current_category_type +'$Feature:Not_Concession'+'$Future:No'

current_public_restrooms['Category'] = current_category_type
current_public_restrooms['SubCategoryName'] = current_category_name
current_public_restrooms['Feature'] = 'Not_Concession'
current_public_restrooms['Future'] = 'No'

Category_locations_for_display[current_category_type] = current_public_restrooms[['Category','SubCategoryName','Feature','Future','geometry']].reset_index(drop=True)

###################################################
# Define access locations to be the same as display locations for Public Restrooms
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Recreation Centers

In [19]:
current_category_type = 'Rec_Center'

###################################################
# Import Current Rec Centers
###################################################

current_rec_centers = Structures[Structures['recreation_center']]
current_rec_centers = current_rec_centers[current_rec_centers['featurestatus'].isin(['Active','Closed Temporarily'])]

current_category_name = 'Cat:'+current_category_type+'$Future:No'

current_rec_centers['Category'] = current_category_type
current_rec_centers['SubCategoryName'] = current_category_name
current_rec_centers['Future'] = 'No'

Category_locations_for_display[current_category_type] = current_rec_centers[['Category','SubCategoryName','Future','geometry']].reset_index(drop=True)

###################################################
# Define access locations to be the same as display locations for Rec Centers
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Tree Stocking (Blocks)

### No Public Data

## Swimming

In [20]:
current_category_type = 'Swimming'

###################################################
# Import Current Pools
###################################################
pools_sql = ('''https://data.cityofnewyork.us/resource/y5rm-wagw.geojson?$select=name,location,polygon&$limit=50000''')
current_pools = gpd.read_file(pools_sql).to_crs(2263)

current_pools['Feature'] = current_pools['location'].astype(str)+'_Pool'
current_pools['Future'] = 'No'
current_pools['Category'] = current_category_type

current_category_name = 'Cat:Swimming'+'$Feature:'+current_pools['Feature'].astype(str)+ '$Future:No'
current_pools['SubCategoryName'] = current_category_name

current_pools = current_pools[['Category','SubCategoryName','Feature','Future','geometry']]
###################################################
###################################################

###################################################
# Import Beaches
###################################################
beach_sql = ('''https://data.cityofnewyork.us/resource/ijwa-mn2v.geojson?$select=name,multipolygon&$limit=50000''')
beaches = gpd.read_file(beach_sql).to_crs(2263)

current_category_name = 'Cat:Swimming'+'$Feature:Beach' + '$Future:No'

beaches['Feature'] = 'Beach'
beaches['Future'] = 'No'
beaches['Category'] = 'Swimming'
beaches['SubCategoryName'] = current_category_name

beaches = beaches[['Category','SubCategoryName','Feature','Future','geometry']]

###################################################
Category_locations_for_display[current_category_type] = pd.concat([current_pools,beaches]).reset_index(drop=True)

###################################################
# Define access locations to be the same as display locations for Public Restrooms
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Tree_Canopy

### No Public Data

## Volunteer Events

In [21]:
current_category_type = 'Volunteer'

###################################################
# Import past and future public education and enrichment events
###################################################
Past_Year_VolunteerEvents_sql = ('''https://www.nycgovparks.org/bigapps/DPR_Volunteer_Events_past_365_days_001.json''')
Past_Year_VolunteerEvents = pd.read_json(Past_Year_VolunteerEvents_sql)
Past_Year_VolunteerEvents['Future'] = 'No'

Future_VolunteerEvents_sql = ('''https://www.nycgovparks.org/bigapps/DPR_Volunteer_Events_upcoming_90_days_001.json''')
Future_VolunteerEvents = pd.read_json(Future_VolunteerEvents_sql)
Future_VolunteerEvents['Future']='Yes'

VolunteerEvents = pd.concat([Past_Year_VolunteerEvents,Future_VolunteerEvents])
VolunteerEvents = gpd.GeoDataFrame(VolunteerEvents, geometry=gpd.points_from_xy(VolunteerEvents['lng'], VolunteerEvents['lat']), crs="EPSG:4326")
VolunteerEvents = VolunteerEvents.to_crs(2263)

current_category_name = 'Cat:Education_Enrichment'+'$Future:'+VolunteerEvents['Future'].astype(str)

VolunteerEvents['Category'] = current_category_type
VolunteerEvents['SubCategoryName'] = current_category_name

Category_locations_for_display[current_category_type] = VolunteerEvents[['Category','SubCategoryName','Future','geometry']].reset_index(drop=True)
###################################################
###################################################


###################################################
# Define access locations to be the same as display locations for volunteer events
###################################################
Category_locations_for_access[current_category_type] = Category_locations_for_display[current_category_type].copy()
###################################################
###################################################

## Waterfronts

### No Public Data

# Upload/save all location data for all categories

In [26]:
All_category_locations = pd.concat(Category_locations_for_access).reset_index(drop=True)

In [ ]:
###################################################
# Upload/save these category locations to SQL server, as a csv, or however you want to store it so that they can be accessed by the third vital parks script
###################################################

# All_category_locations.to_file('Category_locations.geojson', driver='GeoJSON')

# END Vital Parks Script Part 2